# Marvel Knowledge Graph - Neo4j Pipeline

This notebook demonstrates the complete pipeline for building a Marvel Knowledge Graph using:
- **Parallel Extraction**: Extract power origins from characters concurrently
- **Neo4j Integration**: Store graph in Neo4j database
- **Retry Logic**: Automatic retry on failures with tracking
- **Failure Tracking**: Comprehensive logging of successes and failures

## Prerequisites
- Neo4j running (via Docker: `docker-compose up -d neo4j`)
- OpenAI API key in `.env` file
- Dependencies installed (`pip install -r requirements.txt`)

## Step 1: Setup and Imports

Import all necessary modules and set up the environment. This includes:
- Data loading utilities
- Parallel extraction agent
- Neo4j operations
- Graph builder agent

In [20]:
import os
import sys
import asyncio
from pathlib import Path
from dotenv import load_dotenv

# Add project root to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Load environment variables
load_dotenv(project_root.parent / '.env')

# Verify OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in environment!")

print("✓ Environment loaded")
print(f"✓ Project root: {project_root}")
print(f"✓ Neo4j URI: {os.getenv('CONNECTION_URI', 'neo4j://127.0.0.1:7687')}")

✓ Environment loaded
✓ Project root: /Users/hadaszinner/sandbox/marvel_knowledge_grpah
✓ Neo4j URI: neo4j://127.0.0.1:7687


In [21]:
# Import project modules
from src.utils.data_loader import get_sample_characters, load_all_characters
from src.agents.parallel_extraction import extract_batch_parallel
from src.agents.graph_builder_agent import GraphBuilderAgent
from src.graph.neo4j_operations import Neo4jOperations

print("✓ All modules imported successfully")

✓ All modules imported successfully


## Step 2: Connect to Neo4j

Establish connection to Neo4j database. This will:
- Connect using credentials from `.env` file
- Verify connectivity
- Create necessary constraints for data integrity

In [22]:
# Initialize Neo4j connection
neo4j = Neo4jOperations()

# Create constraints for better performance
neo4j.create_constraints()

# Get current statistics
stats = neo4j.get_statistics()
print(f"\n📊 Current Graph Statistics:")
print(f"   Total Nodes: {stats['total_nodes']}")
print(f"   Total Edges: {stats['total_edges']}")
print(f"   Nodes by Type: {stats['nodes_by_type']}")

✓ Connected to Neo4j at neo4j://127.0.0.1:7687
✓ Constraints created

📊 Current Graph Statistics:
   Total Nodes: 2814
   Total Edges: 4026
   Nodes by Type: {'Character': 392, 'PowerOrigin': 392, 'Power': 1246, 'Significance': 392, 'Validation': 392}


## Step 3: Load Character Data

Load character data from the Marvel dataset. You can choose to:
- Load a small sample for testing (3-10 characters)
- Load a larger batch (100-1000 characters)
- Load all characters (~16,000)

**Note**: Start with a small sample to verify the pipeline works correctly.

In [ ]:
data_path = project_root / "data" / "marvel-wikia-data-with-descriptions.pkl"
N = 500  # Adjust this number
characters = load_all_characters(data_path, use_pickle=True)[:N]
print(f"✓ Loaded first {len(characters)} characters from dataset")

## Step 4: Run Parallel Extraction

Extract power origins and significance for all loaded characters in parallel. This process:
- Runs up to 5 extractions concurrently (configurable)
- Automatically retries failed extractions (up to 3 times)
- Tracks all failures with detailed error messages
- Shows progress bar during extraction
- Continues even if some characters fail

**Expected duration**: ~5-10 seconds per character (faster with parallel processing)

In [ ]:
# Configure extraction parameters
MAX_CONCURRENT = 12  # Number of concurrent extractions
MAX_RETRIES = 3     # Retry attempts per character
OUTPUT_DIR = project_root / "data" / "processed"

# Run parallel extraction
extraction_summary = await extract_batch_parallel(
    characters=characters,
    max_concurrent=MAX_CONCURRENT,
    max_retries=MAX_RETRIES,
    output_dir=OUTPUT_DIR,
    verbose=True
)

print(f"\n{'='*80}")
print("EXTRACTION COMPLETE")
print(f"{'='*80}")
print(f"Success Rate: {extraction_summary.successful}/{extraction_summary.total_characters} "
      f"({extraction_summary.successful/extraction_summary.total_characters*100:.1f}%)")

## Step 5: Review Extraction Results

Examine the extraction results:
- View successful extractions
- Check failed extractions (if any)
- Identify characters that need manual review

In [ ]:
# Display successful extractions summary
print("\n✓ SUCCESSFUL EXTRACTIONS:\n")
for result in extraction_summary.successful_extractions[:5]:  # Show first 5
    ext = result.extraction
    print(f"📌 {ext.character_name}")
    print(f"   Origin Type: {ext.power_origin.type.value}")
    print(f"   Confidence: {ext.power_origin.confidence.value}")
    print(f"   Impact: {ext.significance.impact_level.value}")
    print(f"   Attempts: {result.attempts}, Duration: {result.duration_seconds:.2f}s")
    print()

if len(extraction_summary.successful_extractions) > 5:
    print(f"... and {len(extraction_summary.successful_extractions) - 5} more")

In [ ]:
# Display failed extractions (if any)
if extraction_summary.failed_extractions:
    print("\n✗ FAILED EXTRACTIONS:\n")
    for result in extraction_summary.failed_extractions:
        print(f"❌ {result.character_name} (ID: {result.character_id})")
        print(f"   Error: {result.error_message}")
        print(f"   Attempts: {result.attempts}")
        print()
else:
    print("\n✓ No failed extractions!")

## Step 6: Clear Neo4j Database (Optional)

**Warning**: This will delete all existing data in Neo4j!

Uncomment and run this cell if you want to start fresh.

In [ ]:
# Uncomment to clear database
# neo4j.clear_database()
# print("✓ Database cleared")

## Step 7: Build Knowledge Graph in Neo4j

Create nodes and relationships in Neo4j for all successfully extracted characters. This process:
- Creates Character nodes
- Creates PowerOrigin nodes
- Creates Power nodes for each capability
- Creates Significance nodes
- Creates Validation nodes
- Links everything with appropriate relationships

In [ ]:
# Get character data for successful extractions
successful_char_ids = {r.character_id for r in extraction_summary.successful_extractions}
successful_characters = [c for c in characters if c.page_id in successful_char_ids]

print(f"\nBuilding graph for {len(successful_characters)} characters...\n")

# Build graph for each character
for result in extraction_summary.successful_extractions:
    # Find matching character
    character = next((c for c in successful_characters if c.page_id == result.character_id), None)
    
    if not character:
        print(f"⚠️  Could not find character data for {result.character_name}")
        continue
    
    extraction = result.extraction
    
    print(f"Building graph for: {character.name}")
    
    # 1. Create Character node
    char_id = neo4j.add_character_node(
        name=character.name,
        page_id=character.page_id,
        alignment=character.align,
        appearances=character.appearances
    )
    
    # 2. Create PowerOrigin node
    origin_id = neo4j.add_power_origin_node(
        character_id=char_id,
        origin_type=extraction.power_origin.type.value,
        description=extraction.power_origin.description,
        confidence=extraction.power_origin.confidence.value,
        evidence=extraction.power_origin.evidence
    )
    
    # 3. Link character to origin
    neo4j.add_relationship(char_id, origin_id, "HAS_ORIGIN")
    
    # 4. Create Power nodes and relationships
    for capability in extraction.significance.unique_capabilities:
        power_id = neo4j.add_power_node(power_name=capability)
        neo4j.add_relationship(char_id, power_id, "POSSESSES_POWER")
        neo4j.add_relationship(origin_id, power_id, "CONFERS")
    
    # 5. Create Significance node
    sig_id = neo4j.add_significance_node(
        character_id=char_id,
        why_matters=extraction.significance.why_matters,
        impact_level=extraction.significance.impact_level.value,
        strategic_value=extraction.significance.strategic_value
    )
    
    # 6. Link character to significance
    neo4j.add_relationship(char_id, sig_id, "HAS_SIGNIFICANCE")
    
    # 7. Create Validation node
    val_id = neo4j.add_validation_node(
        character_id=char_id,
        is_valid=True,
        confidence_score=1.0,
        completeness_score=1.0
    )
    
    # 8. Link validation
    neo4j.add_relationship(char_id, val_id, "EXTRACTION_VALIDATED")
    
    print(f"  ✓ {character.name} - {len(extraction.significance.unique_capabilities)} powers")

print(f"\n✓ Graph construction complete!")

## Step 8: Verify Graph Statistics

Check the Neo4j graph to see what was created:
- Total nodes and relationships
- Breakdown by node type
- Breakdown by relationship type

In [23]:
# Get updated statistics
stats = neo4j.get_statistics()

print("\n" + "="*80)
print("NEO4J GRAPH STATISTICS")
print("="*80)
print(f"\n📊 Overview:")
print(f"   Total Nodes: {stats['total_nodes']}")
print(f"   Total Relationships: {stats['total_edges']}")

print(f"\n📦 Nodes by Type:")
for node_type, count in sorted(stats['nodes_by_type'].items()):
    print(f"   {node_type}: {count}")

print(f"\n🔗 Relationships by Type:")
for rel_type, count in sorted(stats['edges_by_type'].items()):
    print(f"   {rel_type}: {count}")

print("\n" + "="*80)


NEO4J GRAPH STATISTICS

📊 Overview:
   Total Nodes: 2814
   Total Relationships: 4026

📦 Nodes by Type:
   Character: 392
   Power: 1246
   PowerOrigin: 392
   Significance: 392
   Validation: 392

🔗 Relationships by Type:
   CONFERS: 1425
   EXTRACTION_VALIDATED: 392
   HAS_ORIGIN: 392
   HAS_SIGNIFICANCE: 392
   POSSESSES_POWER: 1425



## Step 9: Query Individual Characters

Test queries to retrieve character profiles from Neo4j.
Each profile includes:
- Character details
- Power origin information
- List of powers/abilities
- Significance assessment
- Validation status

In [24]:
# Query Spider-Man's profile
profile = neo4j.get_character_profile("Spider-Man (Peter Parker)", search_by="name")

if profile:
    print("\n" + "="*80)
    print(f"CHARACTER PROFILE: {profile['character']['name']}")
    print("="*80)
    
    print(f"\n📋 Basic Info:")
    print(f"   ID: {profile['character']['node_id']}")
    print(f"   Page ID: {profile['character'].get('page_id', 'N/A')}")
    print(f"   Alignment: {profile['character'].get('alignment', 'Unknown')}")
    
    if profile['origin']:
        print(f"\n⚡ Power Origin:")
        print(f"   Type: {profile['origin']['origin_type']}")
        print(f"   Confidence: {profile['origin']['confidence']}")
        print(f"   Description: {profile['origin']['description'][:200]}...")
    
    if profile['powers']:
        print(f"\n💪 Powers ({len(profile['powers'])}):")
        for power in profile['powers']:
            print(f"   - {power['name']}")
    
    if profile['significance']:
        print(f"\n🌟 Significance:")
        print(f"   Impact Level: {profile['significance']['impact_level']}")
        print(f"   Why It Matters: {profile['significance']['why_matters'][:200]}...")
else:
    print("Character not found in Neo4j")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: VALIDATED_BY)} {position: line: 6, column: 30, offset: 250} for query: '\n        MATCH (c:Character {name: $identifier})\n        OPTIONAL MATCH (c)-[:HAS_ORIGIN]->(o:PowerOrigin)\n        OPTIONAL MATCH (o)-[:CONFERS]->(p:Power)\n        OPTIONAL MATCH (c)-[:HAS_SIGNIFICANCE]->(s:Significance)\n        OPTIONAL MATCH (c)-[:VALIDATED_BY]->(v:Validation)\n        RETURN c, o, collect(DISTINCT p) as powers, s, v\n        '



CHARACTER PROFILE: Spider-Man (Peter Parker)

📋 Basic Info:
   ID: character_1678
   Page ID: 1678
   Alignment: Good Characters

⚡ Power Origin:
   Type: accident
   Confidence: high
   Description: Peter Parker gained his powers after being bitten by a radioactive spider, which altered his physiology and granted him superhuman abilities....

💪 Powers (9):
   - superhuman strength
   - wall-crawling
   - spider-sense
   - web-slinging
   - enhanced agility
   - Wall-crawling
   - Superhuman strength
   - Spider-sense (danger detection)
   - Web-slinging (using web-shooters)

🌟 Significance:
   Impact Level: local
   Why It Matters: Spider-Man's powers allow him to protect his city and fight against various threats, making him a key figure in the superhero community. His abilities also resonate with themes of responsibility and h...


## Step 10: List All Characters

Get a list of all characters currently in the Neo4j database.

In [25]:
# List all characters
all_characters = neo4j.list_all_characters(limit=200)

print(f"\nCharacters in Database ({len(all_characters)}):")
print("="*80)
for char in all_characters:
    alignment = char.get('alignment', 'Unknown')
    appearances = char.get('appearances', 'N/A')
    print(f"  - {char['name']} ({alignment}, {appearances} appearances)")


Characters in Database (200):
  - Abner Jenkins (Earth-616) (Good Characters, 284 appearances)
  - Adam Warlock (Earth-616) (Good Characters, 252 appearances)
  - Adolf Hitler (Earth-616) (Bad Characters, 145 appearances)
  - Adrian Toomes (Earth-616) (Bad Characters, 248 appearances)
  - Agatha Harkness (Earth-616) (Good Characters, 122 appearances)
  - Akihiro (Earth-616) (Bad Characters, 189 appearances)
  - Aleksei Sytsevich (Earth-616) (Bad Characters, 239 appearances)
  - Alexander Power (Earth-616) (Good Characters, 197 appearances)
  - Alexander Summers (Earth-616) (Good Characters, 558 appearances)
  - Alicia Masters (Earth-616) (Good Characters, 293 appearances)
  - Alison Blaire (Earth-616) (Good Characters, 353 appearances)
  - Amadeus Cho (Earth-616) (Good Characters, 103 appearances)
  - Amara Aquilla (Earth-616) (Good Characters, 217 appearances)
  - Amora (Earth-616) (Bad Characters, 247 appearances)
  - Angelica Jones (Earth-616) (Good Characters, 319 appearances)
  -

## Step 11: Run Custom Cypher Queries

Execute custom Cypher queries against Neo4j. Examples:
- Find characters by origin type
- Find characters with specific powers
- Analyze power distribution

In [26]:
# Example: Find all characters with 'accident' origin type
query = """
MATCH (c:Character)-[:HAS_ORIGIN]->(o:PowerOrigin {origin_type: 'accident'})
RETURN c.name as character_name, o.description as origin_description
"""

with neo4j.driver.session() as session:
    result = session.run(query)
    records = list(result)
    
    print(f"\nCharacters with 'accident' origin ({len(records)}):")
    print("="*80)
    for record in records:
        print(f"  - {record['character_name']}")
        print(f"    {record['origin_description'][:150]}...")
        print()


Characters with 'accident' origin (23):
  - Spider-Man (Peter Parker)
    Peter Parker gained his powers after being bitten by a radioactive spider, which altered his physiology and granted him superhuman abilities....

  - Hulk (Robert Bruce Banner)
    Bruce Banner was exposed to gamma radiation during an experiment gone wrong, which caused his transformation into the Hulk....

  - Matthew Murdock (Earth-616)
    Matthew Murdock was blinded at a young age in a chemical accident that also gave him special abilities like superhuman senses, agility, reflexes, stam...

  - Norman Osborn (Earth-616)
    Norman Osborn was accidentally exposed to a chemical formula that increases his strength and intelligence, leading to his transformation into the Gree...

  - Richard Jones (Earth-616)
    Rick Jones inadvertently triggered the Gamma Bomb test that transformed Dr. Bruce Banner into the Hulk, which led to his involvement in the fallout an...

  - Human Torch (Android) (Earth-616)
    The H

In [ ]:
# Example: Count characters by impact level
query = """
MATCH (c:Character)-[:HAS_SIGNIFICANCE]->(s:Significance)
RETURN s.impact_level as impact_level, count(c) as character_count
ORDER BY character_count DESC
"""

with neo4j.driver.session() as session:
    result = session.run(query)
    records = list(result)
    
    print(f"\nCharacters by Impact Level:")
    print("="*80)
    for record in records:
        print(f"  {record['impact_level']:10s}: {record['character_count']} characters")


Characters by Impact Level:
  global    : 189 characters
  regional  : 129 characters
  cosmic    : 45 characters
  local     : 29 characters


## Step 12: Natural Language Querying with Agent

Use the Neo4j Query Agent to ask questions in natural language.
The agent will:
- Classify your question type (power origin, abilities, significance, etc.)
- Extract character names from your question
- Retrieve relevant data from Neo4j
- Generate a natural language answer with confidence scores

In [ ]:
# Import the Neo4j Query Agent
from src.agents.neo4j_query_agent import Neo4jQueryAgent

# Create the query agent (reusing the existing neo4j connection)
query_agent = Neo4jQueryAgent(
    neo4j_ops=neo4j,
    llm_model="gpt-4o-mini",
    temperature=0.3,
    verbose=True
)

print("✓ Neo4j Query Agent initialized")

In [ ]:
# Example 1: Ask about a character's power origin
question1 = "How did Spider-Man get his powers?"
result1 = query_agent.query(question1, verbose=True)

print("\n" + "="*80)
print(f"QUESTION: {result1['question']}")
print("="*80)
print(f"\nANSWER:\n{result1['answer']}")
print(f"\nMetadata:")
print(f"  Query Type: {result1['query_type']}")
print(f"  Characters Found: {result1['characters']}")
print(f"  Confidence Level: {result1['confidence_level']}")

In [ ]:
# Example 2: Ask about character abilities
question2 = "What powers does Iron Man have?"
result2 = query_agent.query(question2, verbose=False)

print("\n" + "="*80)
print(f"QUESTION: {result2['question']}")
print("="*80)
print(f"\nANSWER:\n{result2['answer']}")
print(f"\nConfidence: {result2['confidence_level']}")

## Step 12: Cleanup

Close the Neo4j connection when done.

In [ ]:
# Close Neo4j connection
neo4j.close()
print("✓ Neo4j connection closed")

## Summary

This notebook demonstrated:
1. ✅ Parallel character extraction with retry logic
2. ✅ Failure tracking and reporting
3. ✅ Building knowledge graph in Neo4j
4. ✅ Querying characters and relationships
5. ✅ Custom Cypher queries

### Next Steps
- Scale to more characters (adjust batch size in Step 3)
- Add validation agent integration
- Connect API to Neo4j backend
- Visualize graph in Neo4j Browser (http://localhost:7474)